In [44]:
# Exploring sympy as a way of building up the model
from sympy import *
from sympy import vector

In [45]:
# Have to make my own dot, it appears.
import sympy 

class myDotP(Function):
    """
    Delayed dot product.
    """
    def doit(self):
        x,y=self.args
        result=sympy.S.Zero
        
        # standardize both to [n,1]
        if x.shape[1]>1:
            x=x.T
        if y.shape[1]>1:
            y=y.T
        assert x.shape[1]==1
        assert y.shape[1]==1
        assert x.shape[0]==y.shape[0]
        for k in range(x.shape[0]):
            result=result + x[k,0]*y[k,0]
        return result
    
    def _latex(self, printer=None):
        a, b = [printer.doprint(i) for i in self.args]
        return r"%s \cdot %s" % ( a,b )
    
class SumForeach(Sum):
    """Summation over elements in a matrix"""
    
    def __new__(cls, function, *symbols, **assumptions):
        dummy,fset=symbols[0]
        dummy_i=Symbol('idx_'+dummy.name)

        # Allow row or column vector
        if fset.shape[0]==1:
            new_sym=(dummy_i,0,fset.shape[1]-1)
            opand=fset[0,dummy_i]
        elif fset.shape[1]==1:
            new_sym=(dummy_i,0,fset.shape[0]-1)
            opand=fset[dummy_i,0]
            
        new_f=Subs(function,dummy,opand)
        obj = SumForeachExpand(new_f, new_sym, *symbols[1:], **assumptions)
        
        return obj

class SumForeachExpand(Sum):
    # Unevaluated Sum over a matrix
    # One issue here is that it's possible to subs in a matrix
    # which doesn't have the same shape as the one that was used to
    # define the Summation.  That's because the limits are evaluated
    # from fset's shape at the time of instantiation.  Think about it.
    def _latex(self, printer=None):
        # Reach inside the Subs instance:
        dummy=printer.doprint(self.args[0].args[1][0])
        fset=printer.doprint(self.args[0].args[2][0].args[0])
        func=printer.doprint(self.args[0].args[0])
        return r'\sum_{%s \in %s} %s'%(dummy,fset,func)


In [466]:
# TOPOLOGY:
N_f=Symbol("N_f") # global number of faces
N_c=Symbol("N_c") # global number of cells

f,c,j=symbols('f c j') # ,cls=Idx) # Generic face and cell indices
fi,ci=symbols('fi ci') #side index of cell, cell index of side

fw=Wild('f')
jw=Wild('j')
cw=Wild('c')

# Kleptsova uses the sparse s[f,c] connection between edges and cell
# s[f,c] is +1 if face f is adjacent to cell c and f's normal is out of
# c.  -1 if the normal is into c.  0 if f is not adjacent to c.

# I'd rather use a more direct description:
Nf_max=Symbol('N_f,max') # maximum number of sides for a cell
Nf_c=IndexedBase('Nf') # number of sides for cell c

F=IndexedBase('F',shape=(N_c,Nf_max)) # F[c,i] the i'th face adjacent to cell c  
S=IndexedBase('S',shape=(N_c,Nf_max)) # S[c,i] sign of the normal of F[c,i] w.r.t. c's outward normal
# As a MatrixSymbol, had issues with C getting substituted.
C=IndexedBase('C',shape=(N_f,2)) # # opposite of F.  C[f,i] the 0/1 neighboring cell of face f

# Question: what is the right convention here for boundary edges?
c1=IndexedBase('c1') # 'upstream' cell for edge f  duplicates C!
c2=IndexedBase('c2') # 'downstream' cell for edge f

# constant GEOMETRY:
A_c=IndexedBase("A") # cell area
lf=IndexedBase("l") # length of face f
b=IndexedBase('b') # cell bed elevation

dfc=IndexedBase('dfc',shape=(N_c,Nf_max))
df=IndexedBase('df')

#alpha=MatrixSymbol(r'\alpha',Nf_max,2) # weighting for cell c on edge j
alpha=IndexedBase(r'\alpha',shape=(Nf_max,2))

nf=MatrixSymbol(r'\mathbf{n}',Nf_max,2) # edge normal. vector-valued

# variable GEOMETRY:
h_fn=IndexedBase("hf^n") # edge depth
h_cn=IndexedBase('hc^n') # water column depth
eta_n=IndexedBase(r'\eta^n')
eta_np1=IndexedBase(r'\eta^{n+1}')

# DYNAMIC
u_fn=IndexedBase("u^n")       # edge normal velocity
u_fnp1=IndexedBase("u^{n+1}") #
a_fn=IndexedBase("a^n") # advective term

G_fn=IndexedBase("G^n")       # pressure term
G_fnp1=IndexedBase("G^{n+1}") # 
hbar=IndexedBase(r'\overline{h}') # average cell depths in advection
ufstar=MatrixSymbol(r'\mathbf{u^*}',N_f,2) # vector valued
ucstar=MatrixSymbol(r'\mathbf{u_*}',N_c,2) # vector valued


# NUMERICAL
g=Symbol('g') # gravity
dt=Symbol("\Delta t")
theta=Symbol(r"\theta")

In [467]:
# Kleptsova, eq 5, rewritten with S,F instead of sfc, and combine the
# summations for clarity
eq5a=Eq( A_c[c] * eta_np1[c],
         A_c[c] * eta_n[c]  
         - Sum(    dt * S[c,fi] *lf[F[c,fi]] * h_fn[F[c,fi]] 
               * ( theta*u_fnp1[F[c,fi]] + (1-theta)*u_fn[F[c,fi]]),
                           (fi,0,Nf_c[c]-1)) )
eq5a

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*u^{n+1}[F[c, fi]] + (1 - \theta)*u^n[F[c, fi]])*S[c, fi]*hf^n[F[c, fi]]*l[F[c, fi]], (fi, 0, Nf[c] - 1)))

In [468]:
# Kleptsova eq 5, omitting Coriolis, and simplified for single layer.
# Use fw=wildcard f to simplify substitution below
eq5b=Eq( u_fnp1[fw],
        u_fn[fw] + dt*a_fn[fw]
        -g*dt*(theta*G_fnp1[fw] + (1-theta)*G_fn[fw]) )
eq5b

Eq(u^{n+1}[f_], -\Delta t*g*(\theta*G^{n+1}[f_] + (1 - \theta)*G^n[f_]) + \Delta t*a^n[f_] + u^n[f_])

In [469]:
# Definition of h_fn
# rather than continue with their sfc notation, use the
# more direct c1(f)
eq8a=Eq( h_fn[f],
        Piecewise( (h_cn[c1[f]],  u_fn[f]>0),
                   (h_cn[c2[f]],  u_fn[f]<0),
                   (Max( eta_n[c1[f]], eta_n[c2[f]])
                    -
                    Max( b[c1[f]], b[c2[f]]),
                    True) ) )
#display(eq8a)
eq8b=Eq( h_cn[c],
         eta_n[c] - b[c])
#display(eq8b)

eq8=eq8a.replace(eq8b.lhs,eq8b.rhs)
eq8                

Eq(hf^n[f], Piecewise((hc^n[c1[f]], u^n[f] > 0), (hc^n[c2[f]], u^n[f] < 0), (Max(\eta^n[c1[f]], \eta^n[c2[f]]) - Max(b[c1[f]], b[c2[f]]), True)))

In [470]:
# The solution procedure
# eq5a give the freesurface update.  But it has the implicit velocity
# in it.  So we substitute, but use replace because the indexes 
# defeat subs().
eq_comb1=eq5a.replace(eq5b.lhs,eq5b.rhs)
eq_comb1

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*(-\Delta t*g*(\theta*G^{n+1}[F[c, fi]] + (1 - \theta)*G^n[F[c, fi]]) + \Delta t*a^n[F[c, fi]] + u^n[F[c, fi]]) + (1 - \theta)*u^n[F[c, fi]])*S[c, fi]*hf^n[F[c, fi]]*l[F[c, fi]], (fi, 0, Nf[c] - 1)))

In [471]:
# Define G operator
# in 3D this would also include baroclinic, which would
# be explicit
eqG=Eq( G_fn[fw],
        (eta_n[c2[fw]]-eta_n[c1[fw]])/df[fw])
eqGn=eqG
eqGnp1=eqG.subs(eta_n,eta_np1).subs(G_fn,G_fnp1)
display(eqGn)
eqGnp1

Eq(G^n[f_], (-\eta^n[c1[f_]] + \eta^n[c2[f_]])/df[f_])

Eq(G^{n+1}[f_], (-\eta^{n+1}[c1[f_]] + \eta^{n+1}[c2[f_]])/df[f_])

In [472]:
eq_comb2=eq_comb1.replace(eqGn.lhs,eqGn.rhs)
eq_comb2=eq_comb2.replace(eqGnp1.lhs,eqGnp1.rhs)
eq_comb2=eq_comb2.replace(a_fn[fw],0) # while testing, drop advection
eq_comb2

Eq(A[c]*\eta^{n+1}[c], A[c]*\eta^n[c] - Sum(\Delta t*(\theta*(-\Delta t*g*(\theta*(-\eta^{n+1}[c1[F[c, fi]]] + \eta^{n+1}[c2[F[c, fi]]])/df[F[c, fi]] + (1 - \theta)*(-\eta^n[c1[F[c, fi]]] + \eta^n[c2[F[c, fi]]])/df[F[c, fi]]) + u^n[F[c, fi]]) + (1 - \theta)*u^n[F[c, fi]])*S[c, fi]*hf^n[F[c, fi]]*l[F[c, fi]], (fi, 0, Nf[c] - 1)))

In [475]:
# Now make some of those concrete:

# 3 cells in a row.
# two faces.

# Inputs:

# Dimensions are handled separately because they can appear 
# as args for later symbols.
dims={
     N_f: 2, # global number of faces
     N_c: 3, # global number of cells
     Nf_max: 2, # maximum number of sides for a cell
}

inp={theta: 0.55,
     dt: 1.0,
     g: 9.8, # gravity
     Nf_c:Array([1,2,1]), # number of sides for cell c
     F:Array( [[0,-1], # F[c,i] the i'th face adjacent to cell c
                [0,1],
               [1,-1]]),  
     C:Array( [ [0,1],[1,2]]), # TODO merge w/ c1,c2
     S:Array( [[1,0], # S[c,i] sign of the normal of F[c,i] w.r.t. c's outward normal
                [-1,1],
                [-1,0]] ), 
     c1:Array( [0,1]), # 'upstream' cell for edge f
     c2:Array( [1,2]), # 'downstream' cell for edge f
     A_c:Array( [1.0,1.0,1.0]), # cell area
     lf:Array( [1,1] ), # length of face f
     b:Array( [0,0,0] ), # cell bed elevation
     df:Array( [1.0,1.0]),
     dfc:Array( [[0.5,oo],[0.5,0.5],[0.5,oo]]),
     alpha:Array( [[0.5,0.5],
                   [0.5,0.5]]),
     nf:Matrix([[1.0,0], # Needs to be Matrix so it can be sliced.
                [1.0,0]])
    }

# Some of the inputs are dimensions for the other inputs
# update those symbolic dimensions with real dimensions now
# otherwise the symbolic vs concrete dimensions prevent substitutions
# from working.

state={
     # h_fn is computed # edge depth
     # h_cn is computed # water column depth
     eta_n: Array([1.1,1,1]),
     # eta_np1 is computed 

     u_fn: Array([0.0,0.0]) # edge normal velocity
     # u_fnp1 is computed 
     # a_fn is computed # advective term
}

def sub_all(expr,verbose=False):
    # For something like F, which is defined with symbolic 
    # dimensions but is subject to substitution with
    # concrete dimensions, need to allow for both.
    # assume that all dimensions are known ahead of time, and
    # occur at the start of defs, so any occurrence in expr 
    # will have concrete dimensions.
    for defs in [dims,inp,state]:
        for k in defs:
            if k not in dims:
                ks=k.subs(dims) # Everybody is concrete
            else:
                ks=k
            # if k==expr: continue # ?
            if verbose: print('subbing ',k)
            #try:
            expr=expr.subs(ks,defs[k])
            #except Exception as exc:
            #    print("Sub of %s failed"%k)
            if verbose: print(srepr(expr))
    # eventually the parts that depend on state might be split off
    return expr

def sub_eq(expr,verbose=False):
    return Eq(sub_all(expr.lhs,verbose=verbose),
              sub_all(expr.rhs,verbose=verbose))

In [476]:
# For composite types like Matrix that can have symbolic dimensions,
# here compile synonyms with those dimensions made concrete.

# syn={}

# for s in [inp,state]:
#     updates={}
#     for k in s:
#         if isinstance(k,MatrixSymbol):
#             old_sym=k
#             new_sym=MatrixSymbol(old_sym.args[0],*[Wild('w%d'%wi) for wi,w in enumerate(old_sym.args[1:])])
#             updates[k]=new_sym
#         elif isinstance(k,IndexedBase):
#             old_sym=k
#             new_sym=sub_all(old_sym)
#             updates[k]=new_sym
#     for k in updates:
#         old=k
#         new=updates[k]
#         s[new]=s.pop(old)
    

In [477]:
h_cn_real=Array( [sub_all(eq8b.rhs.subs(c,i)) for i in range(dims[N_c])])
state[h_cn]=h_cn_real
h_cn_real

[1.10000000000000, 1, 1]

In [478]:
# Precalculate h_fn
eq8_0=sub_all(eq8.rhs)
h_fn_real=Array( [ eq8_0.subs(f,i) for i in range(dims[N_f])] )
state[h_fn]=h_fn_real
h_fn_real

[1.10000000000000, 1]

In [479]:
# Note that if eta_np1 has already been computed, this
# will evaluate to ~0 since the solution will get substituted
# in.
eq_real=sub_all(eq_comb2.rhs - eq_comb2.lhs)

cell_eqs=[  eq_real.subs(c,i).doit()
          for i in range(dims[N_c]) ]
#cell_eqs=[ e.rhs-e.lhs for e in cell_eqs]
unknowns=[eta_np1[i] for i in range(dims[N_c])]
for e in cell_eqs:
    display(e)

-4.26095*\eta^{n+1}[0] + 3.26095*\eta^{n+1}[1] + 0.833195

3.26095*\eta^{n+1}[0] - 7.22545*\eta^{n+1}[1] + 2.9645*\eta^{n+1}[2] + 1.266805

2.9645*\eta^{n+1}[1] - 3.9645*\eta^{n+1}[2] + 1.0

In [480]:
if eta_np1 not in state:
    eta_set=linsolve(cell_eqs,unknowns)
    eta_np1_real=Array(list(eta_set)[0])
    state[eta_np1]=eta_np1_real
else:
    print("eta_np1 already computed")
state[eta_np1]

[1.00322743969429, 1.05536943503132, 1.04140312527440]

In [481]:
# And solve for velocity:
eq5c=eq5b.replace(eqGn.lhs,eqGn.rhs).replace(eqGnp1.lhs,eqGnp1.rhs).replace(a_fn[fw],0)
eq5d=sub_all(eq5c)

u_fnp1_real=Array( [eq5d.rhs.subs(fw,i) for i in range(dims[N_f])] )
state[u_fnp1]=u_fnp1_real
u_fnp1_real

[0.159954645133409, 0.0752784095898098]

Advection
---


In [482]:
# finally ucstar
lhs=ucstar[c,0]
#display(lhs)

# Since we'll evaluate this, rather than use it for a substitution,
# do *not* use wilds
rhs=Sum(Subs(1/(A_c[c]*h_cn[c]) * h_fn[f]*lf[f] * u_fn[f] * nf[f,:] * dfc[c,fi],
             f,F[c,fi]),
       (fi,0,Nf_c[c]-1))
#display(rhs)
eq6=Eq(lhs,rhs,evaluate=False) # w/o evaluate=False, this simplifies to False
display(eq6)

# calculating actual ustar
eq6a=eq6.subs(eq8b.lhs,eq8b.rhs)
display(eq6a)

Eq(\mathbf{u_*}[c, 0], Sum(Subs((dfc[c, fi]*hf^n[f]*l[f]*u^n[f]/(A[c]*hc^n[c]))*\mathbf{n}[f:f + 1, :], f, F[c, fi]), (fi, 0, Nf[c] - 1)))

Eq(\mathbf{u_*}[c, 0], Sum(Subs((dfc[c, fi]*hf^n[f]*l[f]*u^n[f]/((\eta^n[c] - b[c])*A[c]))*\mathbf{n}[f:f + 1, :], f, F[c, fi]), (fi, 0, Nf[c] - 1)))

In [483]:
eq6b=sub_eq(eq6a) 

ucstar_vectors=[eq6b.rhs.subs(c,i).doit() 
                for i in range(dims[N_c])]
ucstar_n_real=Matrix( [ [c[0,0],c[0,1]] for c in ucstar_vectors] )

state[ucstar]=ucstar_n_real
ucstar_n_real

Matrix([
[0, 0],
[0, 0],
[0, 0]])

In [484]:
eq_hbar = Eq( hbar[j], 
             Sum( alpha[j,ci] * h_cn[C[ci,j]],
                   (ci,0,1)))
eq_hbar

Eq(\overline{h}[j], Sum(\alpha[j, ci]*hc^n[C[ci, j]], (ci, 0, 1)))

In [485]:
# calculate hbar
hbar_real = Array( [sub_all(eq_hbar.rhs.subs(j,jj)).doit()
                    for jj in range(dims[N_f])])
state[hbar]=hbar_real
hbar_real

[1.05000000000000, 1.00000000000000]

In [486]:
# Define the cell-centered part, then pull together the edge centered part.
ac_n=MatrixSymbol(r'\tilde{a}^n',N_c,N_f) # cell c advection w.r.t. face f

# Since this is getting subbed into the edge equation below, write in
# terms of wilds.
eq18_cell = Eq( ac_n[cw,jw],
    Sum( Subs(  ( myDotP(nf[jw,:], ufstar[f,:]) - u_fn[jw]) 
                 * h_fn[f] * lf[f] * S[fi,cw] * u_fn[f]
                 /(A_c[cw] * hbar[jw]),
              f,F[fi,cw]),
        (fi,0,Nf_c[cw]-1)) )

eq18_cell

Eq(\tilde{a}^n[c_, j_], Sum(Subs(S[fi, c_]*hf^n[f]*l[f]*u^n[f]*(-u^n[j_] + myDotP(\mathbf{n}[j_:j_ + 1, :], \mathbf{u^*}[f:f + 1, :]))/(A[c_]*\overline{h}[j_]), f, F[fi, c_]), (fi, 0, Nf[c_] - 1)))

In [487]:
eq18_face = Eq( a_fn[j],
               Sum( alpha[j,ci] * ac_n[C[j,ci],j],
                    (ci,0,1) ) )
eq18_face

Eq(a^n[j], Sum(\alpha[j, ci]*\tilde{a}^n[C[j, ci], j], (ci, 0, 1)))

In [488]:
eq18b=eq18_face.replace(eq18_cell.lhs, eq18_cell.rhs)
eq18b

Eq(a^n[j], Sum(\alpha[j, ci]*Sum(Subs(S[fi, C[j, ci]]*hf^n[f]*l[f]*u^n[f]*(-u^n[j] + myDotP(\mathbf{n}[j:j + 1, :], \mathbf{u^*}[f:f + 1, :]))/(A[C[j, ci]]*\overline{h}[j]), f, F[fi, C[j, ci]]), (fi, 0, Nf[C[j, ci]] - 1)), (ci, 0, 1)))

In [489]:
# and ustar...
#
# gets a little dirty -- u* . n == uj when 
# if the upwind cell of f is the cell containing j
# in that case the flow is out of the 2-cell CV of
# j

# This does not get precomputed, but instead substituted,
# so use wilds
ja=Wild('j_a')
fa=Wild('f_a')

lhs=myDotP(nf[ja,:],ufstar[fa,:])

rhs=Piecewise( 
    (u_fn[ja], And(u_fn[fa]>0,Or(Eq(c1[fa],c1[ja]),
                               Eq(c1[fa],c2[ja])))),
    (u_fn[ja], And(u_fn[fa]<=0,Or(Eq(c2[fa],c1[ja]),
                                Eq(c2[fa],c2[ja])))),
    (myDotP(nf[ja,:],ucstar[c1[fa],:]),  u_fn[fa]>0),
    (myDotP(nf[ja,:],ucstar[c2[fa],:]),  True) )

eq_ufstar=Eq(lhs,rhs,evaluate=False)
eq_ufstar

Eq(myDotP(\mathbf{n}[j_a_:j_a_ + 1, :], \mathbf{u^*}[f_a_:f_a_ + 1, :]), Piecewise((u^n[j_a_], ((u^n[f_a_] <= 0) | Eq(c1[f_a_], c1[j_a_]) | Eq(c1[f_a_], c2[j_a_])) & ((u^n[f_a_] > 0) | Eq(c1[j_a_], c2[f_a_]) | Eq(c2[f_a_], c2[j_a_])) & (Eq(c1[f_a_], c1[j_a_]) | Eq(c1[f_a_], c2[j_a_]) | Eq(c1[j_a_], c2[f_a_]) | Eq(c2[f_a_], c2[j_a_]))), (myDotP(\mathbf{n}[j_a_:j_a_ + 1, :], \mathbf{u_*}[c1[f_a_]:c1[f_a_] + 1, :]), u^n[f_a_] > 0), (myDotP(\mathbf{n}[j_a_:j_a_ + 1, :], \mathbf{u_*}[c2[f_a_]:c2[f_a_] + 1, :]), True)))

In [490]:
eq18c=eq18b.replace(eq_ufstar.lhs,eq_ufstar.rhs)
eq18c

Eq(a^n[j], Sum(\alpha[j, ci]*Sum(Subs(S[fi, C[j, ci]]*hf^n[f]*l[f]*u^n[f]*(-u^n[j] + Piecewise((u^n[j], ((u^n[f] <= 0) | Eq(c1[f], c1[j]) | Eq(c1[f], c2[j])) & ((u^n[f] > 0) | Eq(c1[j], c2[f]) | Eq(c2[f], c2[j])) & (Eq(c1[f], c1[j]) | Eq(c1[f], c2[j]) | Eq(c1[j], c2[f]) | Eq(c2[f], c2[j]))), (myDotP(\mathbf{n}[j:j + 1, :], \mathbf{u_*}[c1[f]:c1[f] + 1, :]), u^n[f] > 0), (myDotP(\mathbf{n}[j:j + 1, :], \mathbf{u_*}[c2[f]:c2[f] + 1, :]), True)))/(A[C[j, ci]]*\overline{h}[j]), f, F[fi, C[j, ci]]), (fi, 0, Nf[C[j, ci]] - 1)), (ci, 0, 1)))

In [491]:
eq18d=sub_eq(eq18c)

eq18d

Eq(a^n[j], Sum([[0.500000000000000, 0.500000000000000], [0.500000000000000, 0.500000000000000]][j, ci]*Sum(Subs([0.0, 0.0][f]*[1, 1][f]*[1.10000000000000, 1][f]*[[1, 0], [-1, 1], [-1, 0]][fi, [[0, 1], [1, 2]][j, ci]]*(-[0.0, 0.0][j] + Piecewise(([0.0, 0.0][j], (([0.0, 0.0][f] <= 0) | Eq([0, 1][f], [0, 1][j]) | Eq([0, 1][f], [1, 2][j])) & (([0.0, 0.0][f] > 0) | Eq([0, 1][j], [1, 2][f]) | Eq([1, 2][f], [1, 2][j])) & (Eq([0, 1][f], [0, 1][j]) | Eq([0, 1][f], [1, 2][j]) | Eq([0, 1][j], [1, 2][f]) | Eq([1, 2][f], [1, 2][j]))), (myDotP(Matrix([
[1.0, 0],
[1.0, 0]])[j:j + 1, :], Matrix([
[0, 0],
[0, 0],
[0, 0]])[[0, 1][f]:[0, 1][f] + 1, :]), [0.0, 0.0][f] > 0), (myDotP(Matrix([
[1.0, 0],
[1.0, 0]])[j:j + 1, :], Matrix([
[0, 0],
[0, 0],
[0, 0]])[[1, 2][f]:[1, 2][f] + 1, :]), True)))/([1.00000000000000, 1.00000000000000, 1.00000000000000][[[0, 1], [1, 2]][j, ci]]*[1.05000000000000, 1.00000000000000][j]), f, [[0, -1], [0, 1], [1, -1]][fi, [[0, 1], [1, 2]][j, ci]]), (fi, 0, [1, 2, 1][[[0, 1], [1,

In [494]:
e1=eq18d.subs(j,0)
e1

Eq(a^n[0], Sum([[0.500000000000000, 0.500000000000000], [0.500000000000000, 0.500000000000000]][0, ci]*Sum(Subs(0.952380952380952*[0.0, 0.0][f]*[1, 1][f]*[1.10000000000000, 1][f]*[[1, 0], [-1, 1], [-1, 0]][fi, [[0, 1], [1, 2]][0, ci]]*Piecewise((0.0, (Eq([0, 1][f], 0) | Eq([0, 1][f], 1) | ([0.0, 0.0][f] <= 0)) & (Eq([1, 2][f], 0) | Eq([1, 2][f], 1) | ([0.0, 0.0][f] > 0)) & (Eq([0, 1][f], 0) | Eq([0, 1][f], 1) | Eq([1, 2][f], 0) | Eq([1, 2][f], 1))), (myDotP(Matrix([
[1.0, 0],
[1.0, 0]])[:1, :], Matrix([
[0, 0],
[0, 0],
[0, 0]])[[0, 1][f]:[0, 1][f] + 1, :]), [0.0, 0.0][f] > 0), (myDotP(Matrix([
[1.0, 0],
[1.0, 0]])[:1, :], Matrix([
[0, 0],
[0, 0],
[0, 0]])[[1, 2][f]:[1, 2][f] + 1, :]), True))/[1.00000000000000, 1.00000000000000, 1.00000000000000][[[0, 1], [1, 2]][0, ci]], f, [[0, -1], [0, 1], [1, -1]][fi, [[0, 1], [1, 2]][0, ci]]), (fi, 0, [1, 2, 1][[[0, 1], [1, 2]][0, ci]] - 1)), (ci, 0, 1)))

In [495]:
# So close. But this is failing with the NoneType no attribute groups
# error.  Usually that has been an Array/Matrix mismatch.
e1.doit() # HERE

AttributeError: 'NoneType' object has no attribute 'groups'

Representation of arrays
---

So far I've tried IndexedBase, Matrix and vector. Array is also a possibility.

There were substitution issues with IndexedBase for >1 dimension.

IndexedBase, afaict, cannot be sliced.  Once it gets indexed, it must be indexed over all of its dimensions. On the other hand, at some level it is supposed to support einstein summation notation, so usage for vectors may be possible. That said, I have not seen where this summation convention is actually invoked. It is alluded to in get_contraction_structure, but seems that one would have to manually execute the implied summation.

Matrix is clumsy, but mostly worked. Matrix is always 2D, or an element. This is a bit awkward when what I want is a 2-vector, but I get a 2x1 matrix. While a Vector in a Matrix can get lost, a Matrix seems to be preserved and will evaluate to a zero matrix with shape.  Have to be careful about when the matrix shape has wildcards (good with replace) or has proper dimensions (good with subs). 

In one case, while calculating hbar, there was an issue where C and alpha were not substituted, apparently
because N_f wasn't being replaced in their dimensions. Maybe something about subbing into MatrixElement? Switching to IndexedBase with shape=(N_f,2), and defining with Array seems to help.

Vector is not type consistent. An expression that should be the zero vector may evaluate to scalar zero. This comes up when putting vectors into matrices.  The "type" of the Matrix is not checked and the result is just 0. 

Array can only be concrete.  I.e. it could probably be used to sub in for an IndexedBase, but there is no ArraySymbol.

RecursionError: related to case expr, and call to decompogen.  Adding a check that gens[0]!=f before recurring seems to fix this?

_rec_replace: related to mixing wilds in target expression

none has no attribute group: maybe a matrix error. try subbing in Array


In [ ]:
if 0: # Kleptsova, eq 5, simplified for a single layer,
    # and ignoring the erroneous trailing "= 0"
    sfc=IndexedBase('s')
    eq5a_orig=Eq( A_c[c] * eta_np1[c],
                 A_c[c] * eta_n[c]  
                 - theta*dt * Sum( sfc[f,c]*lf[f] * h_fn[f] * u_fnp1[f],
                               (f,1,N_f))
                 - (1-theta)*dt * Sum( sfc[f,c]*lf[f]*h_fn[f] * u_fn[f],
                                   (f,1,N_f)) )
    eq5a_orig

In [ ]:
if 0: # Original eq18.
    # the 'f' in a_fn is generic.  it will get rendered as a_j^n
    delta=IndexedBase(r'\delta') # ==|sfc|: delta[j,c]=1 if edge j is part of cell c
    j=Symbol('j')
    hbar=IndexedBase(r'\overline{h}')
    ufstar_=IndexedBase(r'\mathbf{u^*}')
    alpha_=IndexedBase(r'\alpha') # weighting for neighbor ci of edge j
    nf_=IndexedBase(r'\mathbf{n}') # edge normal

    N_c=Symbol('N_c')

    eq18_orig = Eq( a_fn[j],
           Sum(delta[j,c] * alpha_[j,c] * 
                      Sum(sfc[f,c]*h_fn[f]*lf[f]/(A_c[c]*hbar[j]) * u_fn[f]*(ufstar_[f]*nf_[j]-u_fn[j]),
                          (f,1,N_f)),
                      (c,  1, N_c) ))
    eq18_orig